In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

#### Read and pre-process data

In [2]:
# fetch data 

football_data = pd.read_csv('CFB2019_subset_inclass.csv')

football_data.head()

,Conference,Games,Off Rank,Off TDs,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Touchdowns,Points Per Game,3rd Percent,Average Time of Possession per Game
0,Mountain West,13,51,55,421.80,28.31,1.20,5.14,298.50,58,34.10,0.54,34:28:00
1,MAC,12,130,14,243.20,52.17,3.83,1.75,47.60,15,10.50,0.26,26:29:00
2,SEC,13,6,76,510.80,65.77,24.14,5.03,168.50,83,47.20,0.52,29:59:00
3,Sun Belt,14,39,65,433.10,64.21,9.57,5.41,231.40,73,38.80,0.45,31:19:00
4,Pac-12,12,30,42,440.10,56.42,6.36,4.63,174.00,42,26.90,0.44,27:24:00


In [3]:
# subset data to only include columns for Normal Sale Condition

len(football_data)

subset_data = football_data.drop(columns = ['Conference','Average Time of Possession per Game'])

subset_data.head()

len(subset_data)

128

,Games,Off Rank,Off TDs,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Touchdowns,Points Per Game,3rd Percent
0,13,51,55,421.80,28.31,1.20,5.14,298.50,58,34.10,0.54
1,12,130,14,243.20,52.17,3.83,1.75,47.60,15,10.50,0.26
2,13,6,76,510.80,65.77,24.14,5.03,168.50,83,47.20,0.52
3,14,39,65,433.10,64.21,9.57,5.41,231.40,73,38.80,0.45
4,12,30,42,440.10,56.42,6.36,4.63,174.00,42,26.90,0.44


128

In [4]:
subset_data.isna().sum()

Games                        0
Off Rank                     0
Off TDs                      0
Off Yards per Game           0
Penalty Yards Per Game       0
Avg Yards Per Punt Return    0
Yards/Rush                   0
Rushing Yards per Game       0
Touchdowns                   0
Points Per Game              0
3rd Percent                  0
dtype: int64

In [5]:
subset_data.columns

Index(['Games', 'Off Rank', 'Off TDs', 'Off Yards per Game',
       'Penalty Yards Per Game', 'Avg Yards Per Punt Return', 'Yards/Rush',
       'Rushing Yards per Game', 'Touchdowns', 'Points Per Game',
       '3rd Percent'],
      dtype='object')

In [6]:
# adding non-linear columns
subset_data['Off_TD_Per_game'] = subset_data['Off TDs']/subset_data['Games']
subset_data['TDs_Per_game'] = subset_data['Touchdowns']/subset_data['Games']

subset_data['Off_yrds_square'] = subset_data['Off Yards per Game']**2
subset_data['Off_yrds_cube'] = subset_data['Off Yards per Game']**3

subset_data['Points _Per_Game_square'] = subset_data['Points Per Game']**2
subset_data['Points _Per_Game_cube'] = subset_data['Points Per Game']**3

subset_data['Off_TD_square'] = subset_data['Off_TD_Per_game']**2
subset_data['Off_TD_cube'] = subset_data['Off_TD_Per_game']**3

subset_data['TD_per_game_square'] = subset_data['TDs_Per_game']**2
subset_data['TD_per_game_cube'] = subset_data['TDs_Per_game']**3
    
subset_data.head()

,Games,Off Rank,Off TDs,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Touchdowns,Points Per Game,...,Off_TD_Per_game,TDs_Per_game,Off_yrds_square,Off_yrds_cube,Points _Per_Game_square,Points _Per_Game_cube,Off_TD_square,Off_TD_cube,TD_per_game_square,TD_per_game_cube
0,13,51,55,421.80,28.31,1.20,5.14,298.50,58,34.10,...,4.23,4.46,"177,915.24","75,044,648.23","1,162.81","39,651.82",17.90,75.73,19.91,88.81
1,12,130,14,243.20,52.17,3.83,1.75,47.60,15,10.50,...,1.17,1.25,"59,146.24","14,384,365.57",110.25,"1,157.62",1.36,1.59,1.56,1.95
2,13,6,76,510.80,65.77,24.14,5.03,168.50,83,47.20,...,5.85,6.38,"260,916.64","133,276,219.71","2,227.84","105,154.05",34.18,199.81,40.76,260.26
3,14,39,65,433.10,64.21,9.57,5.41,231.40,73,38.80,...,4.64,5.21,"187,575.61","81,238,996.69","1,505.44","58,411.07",21.56,100.08,27.19,141.77
4,12,30,42,440.10,56.42,6.36,4.63,174.00,42,26.90,...,3.50,3.50,"193,688.01","85,242,093.20",723.61,"19,465.11",12.25,42.88,12.25,42.88


In [23]:
subset_data.drop(columns = ['Games','Touchdowns','Off TDs'],inplace=True)
subset_data.head()

,Off Rank,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Points Per Game,3rd Percent,Off_TD_Per_game,TDs_Per_game,Off_yrds_square,Off_yrds_cube,Points _Per_Game_square,Points _Per_Game_cube,Off_TD_square,Off_TD_cube,TD_per_game_square,TD_per_game_cube
0,51,421.80,28.31,1.20,5.14,298.50,34.10,0.54,4.23,4.46,"177,915.24","75,044,648.23","1,162.81","39,651.82",17.90,75.73,19.91,88.81
1,130,243.20,52.17,3.83,1.75,47.60,10.50,0.26,1.17,1.25,"59,146.24","14,384,365.57",110.25,"1,157.62",1.36,1.59,1.56,1.95
2,6,510.80,65.77,24.14,5.03,168.50,47.20,0.52,5.85,6.38,"260,916.64","133,276,219.71","2,227.84","105,154.05",34.18,199.81,40.76,260.26
3,39,433.10,64.21,9.57,5.41,231.40,38.80,0.45,4.64,5.21,"187,575.61","81,238,996.69","1,505.44","58,411.07",21.56,100.08,27.19,141.77
4,30,440.10,56.42,6.36,4.63,174.00,26.90,0.44,3.50,3.50,"193,688.01","85,242,093.20",723.61,"19,465.11",12.25,42.88,12.25,42.88


In [24]:
# X_train, X_test, y_train, y_test = train_test_split(subset_data.drop(columns = ['SalePrice']), subset_data['SalePrice'], test_size=0.25)
X_train, X_test, y_train, y_test = train_test_split(subset_data.drop(columns = ['Off Rank']), subset_data['Off Rank'], test_size=0.2, random_state=50)
# Pick a random_state as below and keep using the same number (example 35) to repeat the same test and train data
# X_train, X_test, y_train, y_test = train_test_split(subset_data['bmi'], subset_data['charges'], test_size=0.25, random_state=35)
X_train
X_test
y_train
y_test

,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Points Per Game,3rd Percent,Off_TD_Per_game,TDs_Per_game,Off_yrds_square,Off_yrds_cube,Points _Per_Game_square,Points _Per_Game_cube,Off_TD_square,Off_TD_cube,TD_per_game_square,TD_per_game_cube
39,329.50,61.31,5.67,3.82,144.20,26.70,0.35,2.92,3.38,"108,570.25","35,773,897.38",712.89,"19,034.16",8.54,24.98,11.46,38.77
73,431.40,51.15,8.44,4.96,179.20,36.80,0.40,4.38,4.69,"186,105.96","80,286,111.14","1,354.24","49,836.03",19.22,84.29,22.02,103.31
54,343.40,48.75,13.67,4.78,169.40,25.30,0.32,3.25,3.50,"117,923.56","40,494,950.50",640.09,"16,194.28",10.56,34.33,12.25,42.88
86,273.10,59.08,2.69,3.51,133.80,13.30,0.34,1.50,1.50,"74,583.61","20,368,783.89",176.89,"2,352.64",2.25,3.38,2.25,3.38
77,453.90,47.08,8.20,5.31,229.60,32.50,0.39,3.85,4.00,"206,025.21","93,514,842.82","1,056.25","34,328.12",14.79,56.90,16.00,64.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,327.70,43.77,14.63,4.94,253.20,28.20,0.31,2.92,3.38,"107,387.29","35,190,814.93",795.24,"22,425.77",8.54,24.98,11.46,38.77
109,540.50,73.23,10.85,5.20,223.80,43.40,0.41,5.38,5.69,"292,140.25","157,901,805.12","1,883.56","81,746.50",28.99,156.12,32.40,184.44
11,421.50,55.57,8.93,4.62,166.40,33.60,0.42,4.21,4.36,"177,662.25","74,884,638.38","1,128.96","37,933.06",17.76,74.85,18.98,82.72
96,394.60,73.92,8.56,3.69,154.00,28.30,0.38,3.25,3.42,"155,709.16","61,442,834.54",800.89,"22,665.19",10.56,34.33,11.67,39.88


,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Points Per Game,3rd Percent,Off_TD_Per_game,TDs_Per_game,Off_yrds_square,Off_yrds_cube,Points _Per_Game_square,Points _Per_Game_cube,Off_TD_square,Off_TD_cube,TD_per_game_square,TD_per_game_cube
44,358.90,52.23,10.19,4.30,178.00,29.60,0.43,3.15,3.62,"128,809.21","46,229,625.47",876.16,"25,934.34",9.95,31.37,13.07,47.26
34,408.10,48.93,11.28,4.94,185.10,30.80,0.41,3.36,3.57,"166,545.61","67,967,263.44",948.64,"29,218.11",11.27,37.84,12.76,45.55
12,429.00,49.64,12.04,4.43,167.30,34.70,0.48,4.14,4.50,"184,041.00","78,953,589.00","1,204.09","41,781.92",17.16,71.10,20.25,91.12
84,393.20,37.67,5.58,2.92,83.30,25.80,0.41,3.17,3.25,"154,606.24","60,791,173.57",665.64,"17,173.51",10.03,31.75,10.56,34.33
97,407.70,61.33,19.94,4.80,204.00,30.30,0.47,3.17,3.42,"166,219.29","67,767,604.53",918.09,"27,818.13",10.03,31.75,11.67,39.88
16,443.80,51.38,13.48,4.45,159.10,28.50,0.42,3.46,3.54,"196,958.44","87,410,155.67",812.25,"23,149.12",11.98,41.48,12.52,44.30
110,405.70,55.25,22.50,3.68,150.20,26.70,0.42,3.33,3.50,"164,592.49","66,775,173.19",712.89,"19,034.16",11.11,37.04,12.25,42.88
127,350.90,48.77,10.76,4.86,214.80,25.40,0.43,2.92,3.15,"123,130.81","43,206,601.23",645.16,"16,387.06",8.54,24.98,9.95,31.37
21,528.70,45.60,5.61,6.37,240.40,43.90,0.44,5.67,5.87,"279,523.69","147,784,174.90","1,927.21","84,604.52",32.11,181.96,34.42,201.92
118,388.90,43.71,6.08,3.84,121.20,32.10,0.45,3.64,4.00,"151,243.21","58,818,484.37","1,030.41","33,076.16",13.27,48.34,16.00,64.00


39     115
73      43
54     109
86     129
77      21
      ... 
33     118
109      2
11      52
96      71
48      32
Name: Off Rank, Length: 102, dtype: int64

44     103
34      61
12      48
84      75
97      62
16      28
110     65
127    106
21       5
118     81
63      19
3       39
40      41
123    119
75       4
59      59
7       33
107     53
56      10
8       89
17     117
83      87
38      78
18      37
25     114
42      27
Name: Off Rank, dtype: int64

In [25]:
if True: 
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = pd.DataFrame(sc.fit_transform(X_train), columns = X_train.columns, index = X_train.index)
    X_test = pd.DataFrame(sc.transform(X_test), columns = X_test.columns, index = X_test.index)
    X_train
    X_test
    y_train
    y_test

,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Points Per Game,3rd Percent,Off_TD_Per_game,TDs_Per_game,Off_yrds_square,Off_yrds_cube,Points _Per_Game_square,Points _Per_Game_cube,Off_TD_square,Off_TD_cube,TD_per_game_square,TD_per_game_cube
39,-1.20,0.74,-0.62,-0.80,-0.53,-0.22,-0.77,-0.48,-0.18,-1.16,-1.10,-0.33,-0.39,-0.56,-0.58,-0.30,-0.37
73,0.50,-0.29,0.09,0.80,0.24,1.23,0.12,1.04,1.15,0.45,0.38,1.25,1.19,1.00,0.88,1.13,1.03
54,-0.97,-0.53,1.41,0.55,0.02,-0.42,-1.39,-0.14,-0.06,-0.97,-0.94,-0.51,-0.54,-0.27,-0.35,-0.19,-0.28
86,-2.14,0.51,-1.37,-1.23,-0.76,-2.14,-0.94,-1.96,-2.08,-1.87,-1.61,-1.65,-1.25,-1.48,-1.11,-1.54,-1.14
77,0.88,-0.70,0.02,1.29,1.35,0.61,-0.12,0.48,0.45,0.86,0.82,0.52,0.39,0.35,0.21,0.32,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,-1.23,-1.04,1.65,0.77,1.87,-0.01,-1.65,-0.48,-0.18,-1.19,-1.12,-0.12,-0.22,-0.56,-0.58,-0.30,-0.37
109,2.33,1.95,0.70,1.13,1.22,2.18,0.18,2.08,2.16,2.65,2.96,2.56,2.83,2.42,2.65,2.53,2.79
11,0.34,0.16,0.21,0.32,-0.04,0.77,0.40,0.86,0.81,0.27,0.20,0.70,0.58,0.78,0.65,0.72,0.59
96,-0.11,2.02,0.12,-0.98,-0.32,0.01,-0.29,-0.14,-0.14,-0.18,-0.25,-0.11,-0.21,-0.27,-0.35,-0.27,-0.34


,Off Yards per Game,Penalty Yards Per Game,Avg Yards Per Punt Return,Yards/Rush,Rushing Yards per Game,Points Per Game,3rd Percent,Off_TD_Per_game,TDs_Per_game,Off_yrds_square,Off_yrds_cube,Points _Per_Game_square,Points _Per_Game_cube,Off_TD_square,Off_TD_cube,TD_per_game_square,TD_per_game_cube
44,-0.71,-0.18,0.53,-0.13,0.21,0.20,0.66,-0.24,0.06,-0.74,-0.75,0.07,-0.04,-0.36,-0.42,-0.08,-0.18
34,0.11,-0.52,0.80,0.77,0.37,0.37,0.31,-0.03,0.01,0.04,-0.03,0.25,0.13,-0.16,-0.26,-0.12,-0.22
12,0.46,-0.44,1.00,0.06,-0.02,0.93,1.52,0.79,0.95,0.40,0.33,0.88,0.78,0.69,0.56,0.89,0.77
84,-0.13,-1.66,-0.64,-2.05,-1.88,-0.35,0.23,-0.23,-0.31,-0.21,-0.27,-0.44,-0.49,-0.34,-0.41,-0.42,-0.46
97,0.11,0.74,2.99,0.57,0.79,0.30,1.39,-0.23,-0.14,0.03,-0.04,0.18,0.06,-0.34,-0.41,-0.27,-0.34
16,0.71,-0.27,1.36,0.08,-0.21,0.04,0.55,0.08,-0.02,0.67,0.62,-0.08,-0.18,-0.06,-0.17,-0.15,-0.25
110,0.07,0.13,3.64,-0.99,-0.40,-0.22,0.47,-0.05,-0.06,0.00,-0.07,-0.33,-0.39,-0.19,-0.28,-0.19,-0.28
127,-0.84,-0.53,0.67,0.66,1.03,-0.41,0.70,-0.48,-0.41,-0.86,-0.85,-0.49,-0.53,-0.56,-0.58,-0.50,-0.53
21,2.13,-0.85,-0.63,2.77,1.59,2.25,0.86,2.37,2.34,2.39,2.62,2.66,2.98,2.87,3.29,2.81,3.17
118,-0.21,-1.05,-0.51,-0.77,-1.04,0.55,1.07,0.27,0.45,-0.28,-0.33,0.45,0.33,0.13,-0.00,0.32,0.18


39     115
73      43
54     109
86     129
77      21
      ... 
33     118
109      2
11      52
96      71
48      32
Name: Off Rank, Length: 102, dtype: int64

44     103
34      61
12      48
84      75
97      62
16      28
110     65
127    106
21       5
118     81
63      19
3       39
40      41
123    119
75       4
59      59
7       33
107     53
56      10
8       89
17     117
83      87
38      78
18      37
25     114
42      27
Name: Off Rank, dtype: int64

In [26]:
model = LinearRegression(fit_intercept = True)
# model = Lasso(alpha = 0.1, fit_intercept = True)
#model = Lasso(alpha = 20.0, fit_intercept = True, max_iter=10000)
#model = Ridge(alpha = 0.1, fit_intercept = True)

model.fit(X_train, y_train) 

# The following gives the R-square score
model.score(X_train, y_train) 

# This is the coefficient Beta_1, ..., Beta_7
model.coef_

# This is the coefficient Beta_0
model.intercept_

#linear - 0.99 (r2)
#ridge - 0.97(r2)
#lasso - 0.65(r2)

#linear - 66 (Beta_0)
#ridge - 66(Beta_0)
#lasso - 66(Beta_0)

LinearRegression()

0.9922357203137683

array([ 6.36523052e+02, -4.06881734e-01,  6.20588587e-01,  2.31695187e-01,
       -7.97011982e-01, -1.03061087e+02, -4.25481949e-01, -2.18691640e+01,
        5.73512695e+01, -1.35939202e+03,  6.95784210e+02,  2.59161894e+02,
       -1.61484089e+02,  4.61545572e+01, -2.50724949e+01, -1.68040196e+02,
        1.14570983e+02])

66.01960784313778

In [27]:
test_output = pd.DataFrame(model.predict(X_test), index = X_test.index, columns = ['pred_off_rank'])
# When extending to multiple features remove .array.reshape(-1, 1)
test_output.head()

,pred_off_rank
44,101.42
34,64.00
12,45.22
84,73.84
97,65.08


In [28]:
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_off_rank'] - test_output['Off Rank']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)

,pred_off_rank,Off Rank
44,101.42,103
34,64.00,61
12,45.22,48
84,73.84,75
97,65.08,62


Mean absolute error is 
3.4621009684362654


In [29]:
abs(test_output['pred_off_rank'] - test_output['Off Rank']).mean()/test_output['Off Rank'].mean()
model.score(X_test, y_test)

0.05770168280727109

0.9841847887952706

#### Visualize data

In [15]:
# define function to import viz libraries
import plotly
plotly.offline.init_notebook_mode(connected=True)
from plotly.graph_objs import *
from plotly import tools
import plotly.graph_objects as go
import seaborn as sns

In [16]:
if False: 
    cols = X_train.columns
    for col in cols:
        plot_data = []
        plot_data.append(go.Scatter(x= X_train[col], y= y_train, name = 'Train data actual', mode = 'markers'))
        plot_data.append(go.Scatter(x= X_train[col], y= model.predict(X_train), name = 'Train data predicted', mode = 'markers'))
        layout = go.Layout(xaxis = dict(title=col), yaxis = dict(title= 'SalePrice'), 
                           title = 'Plot of predicted and actual')
        fig = go.Figure(data= plot_data, layout=layout)
        plotly.offline.iplot(fig)

        plot_data = []
        plot_data.append(go.Scatter(x= X_test[col], y= y_test, name = 'Test data actual', mode = 'markers'))
        plot_data.append(go.Scatter(x= X_test[col], y= model.predict(X_test), name = 'Test data predicted', mode = 'markers'))

        layout = go.Layout(xaxis = dict(title=col), yaxis = dict(title= 'SalePrice'), 
                           title = 'Plot of predicted and actual')
        fig = go.Figure(data= plot_data, layout=layout)
        plotly.offline.iplot(fig)


In [17]:
# THIS IS INCORRECT BECAUSE THE INDICES WERE MESSED UP, can you fix these numbers?
# Notes
# Linear model, no regularization
# test score 0.7255, pct error error 30.39%
# Nonlinear model with log, sq and cubes, no regularization
# test score 0.813, pct error error 28.23%
# Linear model, Lasso alpha = 0.1
# test score 0.7255, pct error error 30.39%
# Linear model, Lasso alpha = 10.0
# test score 0.7256, pct error error 30.39%
# Linear model, Ridge alpha = 0.1
# test score 0.7255, pct error error 30.39%
# Linear model, Ridge alpha = 10.0
# test score 0.7267, pct error error 30.31%
# NonLinear model, Lasso alpha = 0.1
# test score 0.813, pct error error 28.23% (user max_iter = 100000, yes 10 times more)
# NonLinear model, Lasso alpha = 10.0
# test score 0.812, pct error error 28.13% (sauser max_iter = 10000, only one feature drop)
# NonLinear model, Ridge alpha = 0.1
# test score 0.813, pct error error 28.15%
# NonLinear model, Ridge alpha = 10.0
# test score 0.8, pct error error 28.79%
# NonLinear model, Lasso alpha = 20.0
# test score 0.811, pct error error 28.08%